In [143]:
import warnings
warnings.filterwarnings("ignore")

In [154]:
import pandas as pd
def pnl(txns, method):
    txns['Date'] = pd.to_datetime(txns['Date'], dayfirst=True, infer_datetime_format=True)
    txns['Profit'] = 0
    
    df = pd.DataFrame()
    tolerance = 1e-10
    for token in list(set(txns['Token'].unique())):
        tk = txns[(txns['Token'] == token)].copy()
        buys = tk[((txns['Type'] == 'trade-buy') | (tk['Type'] == 'buy-in') | (tk['Type'] == 'receive'))].copy() 

        if method == 'FIFO':
            buys = buys.sort_values(by='Date', ascending=False)
        elif method == 'HIFO':
            buys = buys.sort_values(by='Basis', ascending=True)

        sells = tk[((tk['Type'] == 'trade-sell') | (tk['Type'] == 'sell-out'))].copy()
        sells = sells.sort_values(by='Date', ascending=True)

        calcs = {}
        
        for i, s in sells.iterrows():
            date = s['Date']
            wallet = s['Wallet']
            amount = s['Amount']
            buywindow = buys[buys['Date'] <= date]
            buydict = buywindow.to_dict('index')
            while amount > tolerance:
                try:
                    buy = buydict.popitem()
                    leftover = amount - buy[1]['Amount']
                    calcs[0] = {'Date':date, 'Wallet':wallet, 'Token':token}
                    calcs[0]['Sell Basis'] = s['Basis']
                    calcs[0]['Buy Basis'] = buy[1]['Basis']
                    calcs[0]['Buy Date'] = buy[1]['Date']
                    calcs[0]['Buy Lot'] = buy[1]['Amount']
                    calcs[0]['Period'] = s['Date'] - buy[1]['Date']

                    if leftover >= 0:
                        calcs[0]['Amount'] = buy[1]['Amount']
                        profit = calcs[0]['Amount'] * (calcs[0]['Sell Basis'] - calcs[0]['Buy Basis'])
                        calcs[0]['Profit'] = profit
                        amount = leftover
                    else:     
                        calcs[0]['Amount'] = buy[1]['Amount']+leftover
                        profit = calcs[0]['Amount'] * (calcs[0]['Sell Basis'] - calcs[0]['Buy Basis'])
                        calcs[0]['Profit'] = profit
                        buy[1]['Amount'] = -leftover
                        buydict[buy[0]] = buy[1]
                        amount = 0
                    df = pd.concat([df, pd.DataFrame.from_dict(calcs, orient='index')])
                except KeyError:
                    print(f'No more {token} to sell from before {date}.')
                    break
    
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)
    return df

In [157]:
data = pd.read_csv('transactions.csv', names=["Date", 'Wallet', 'Token', 'Amount', 'Basis', 'Value', 'Fee', 'Type'])
methods = ['FIFO', 'HIFO']
for m in methods:
    result = pnl(data, m)
    result.to_csv(f'pnl-2021-{m}.csv')
    print(f'{m} Profit: {result.Profit.sum()}')

No more QTUM to sell from before 2020-02-06 00:00:00.
FIFO Profit: -2044.962917211711
No more QTUM to sell from before 2020-02-06 00:00:00.
HIFO Profit: -7112.939458990609
